<a href="https://colab.research.google.com/github/JamieBali/sudoku/blob/main/HNN_Sudoku_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HNN Sudoku Solver

In [3]:
import numpy as np
import pandas as pd, csv
import math

In [4]:
trivial = [[8,9,1,2,7,0,5,6,3],[6,0,3,1,8,5,9,0,0],[4,5,7,6,3,9,0,0,2],[5,0,6,4,1,7,2,3,0],[7,0,2,9,0,3,8,1,6],[3,1,0,0,2,6,0,5,4],[9,3,8,5,4,0,6,7,0],[1,6,4,0,9,0,3,2,5],[0,7,5,3,6,1,4,9,8]]

# this is a very simple 4x4 sudoku from sudoku.com which we can use to test the network.
four = [[0,3,4,0],[4,0,0,2],[1,0,0,3],[0,2,1,0]]

# these three were taken randomly from sudoku.com
easy = [[0,0,0,0,7,9,0,3,0],[5,0,2,0,6,1,4,7,8],[3,7,6,0,8,5,9,0,2],[0,1,7,5,0,0,8,0,0],[2,0,9,8,3,0,0,0,0],[0,0,0,0,2,0,0,4,0],[0,0,0,0,5,0,2,0,1],[0,2,3,0,0,0,0,5,4],[1,0,0,7,0,0,0,0,0]]
medium = [[0,3,1,0,5,0,0,2,0],[0,0,0,0,0,2,9,0,5],[2,0,0,0,1,0,0,0,0],[3,5,0,0,9,0,0,7,0],[7,0,0,5,0,0,0,4,0],[0,1,0,7,0,3,2,0,0],[1,2,6,3,0,0,0,0,0],[0,9,0,8,0,5,0,0,0],[5,0,0,0,2,0,7,0,0]]
hard = [[0,4,0,0,0,5,0,6,0],[0,0,5,4,2,0,0,0,0],[0,0,1,6,0,3,5,0,4],[0,0,0,0,0,0,7,0,0],[0,3,7,0,0,0,0,1,0],[9,0,0,0,0,4,3,5,0],[0,0,4,2,5,0,0,0,0],[0,0,0,0,0,0,0,7,6],[6,0,9,0,7,0,0,0,5]]

# this is the solved version of the "easy" sudoku above.
solved = [[8,4,1,2,7,9,6,3,5],[5,9,2,3,6,1,4,7,8],[3,7,6,4,8,5,9,1,2],[4,1,7,5,9,6,8,2,3],[2,5,9,8,3,4,1,6,7],[6,3,8,1,2,7,5,4,9],[7,6,4,9,5,3,2,8,1],[9,2,3,6,1,8,7,5,4],[1,8,5,7,4,2,3,9,6]]

# Puzzle Formatting

We need our puzzles to be in binary form, where 1 represents a filled tile and 0 is empty. It needs to be in a 3-dimensional format $i, j, k$ represening, rows, columns, and values seperately, such that

$S[i,j,k] = 1$ where tile $S[i,j] = k$ in a 2-dimensional format.

In [5]:
###
# This converts the [0-9] construction of the puzzle into a [0 | 1] construction which is more suitable for the network. 
###
def network_format(sudoku, size):
  puzzle=[[[0] * size for _ in range(size)]for _ in range(size)]
  for i in range(size):
    for j in range(size):
      if sudoku[i][j] > 0:
        puzzle[i][j][sudoku[i][j]-1] = 1
  return puzzle

###
# This converts the [0| 1] construction of the puzzle back into a [0-9] construction so it can be read. 
###
def readable_format(board, size):
  result = [[0] * size for _ in range(size)]
  for i in range(size):
    for j in range(size):
      for k in range(size):
        if board[i][j][k] == 1:
          result[i][j] = k + 1
  return result

# Hopfield Neurones

We will construct each neurone in the network as its own object and create an array of them to act as the network. This will allow us to make edits to the functionality of the network easier.

In [56]:
class Neuron:
  input=0
  output=0
  energy=0
  alpha=1
  i=0
  j=0
  k=0
  
  def __init__(self, i, j, k, cell_value, size): #constructor to initialize values for each neuron
    self.i = i
    self.j = j
    self.k = k
    self.input = cell_value
    self.size = size

  def step_neurone(self, output_for_each_neuron):    
    i = self.i
    j = self.j
    k = self.k
    first_term = self.input
    second_term = 0.0
    
    ###
    # Domain constraints - increase energy with violations to rules of sudoku
    ###
    for i_itt in range(0,self.size): # in the same row
      if(i_itt!=i):                  # but not self
        second_term += output_for_each_neuron[i_itt][j][k]

    for j_itt in range(0,self.size): # in the same column
      if(j_itt!=j):                  # but not self
         second_term += output_for_each_neuron[i][j_itt][k]

    for k_itt in range(0,self.size): # in the same tile
      if(k_itt!=k):                  # but not self
        second_term += output_for_each_neuron[i][j][k_itt]
    
    sub_size = int(math.sqrt(self.size))
    sub_num = (math.floor(i/sub_size) * sub_size) + (math.floor(j/sub_size))
    r_start = math.floor(sub_num / sub_size)  * sub_size
    c_start = (sub_num % sub_size) * sub_size

    # 0 | 3 | 6
    # ---------
    # 1 | 4 | 7
    # ---------
    # 2 | 5 | 8
    
    for i_itt in range(r_start,r_start+sub_size): # for each tile in the sub-grid
      for j_itt in range(c_start, c_start+sub_size):
        if not (i_itt==i and j_itt==j): # but not self
          second_term += output_for_each_neuron[i_itt][j_itt][k]
          
    self.energy = first_term + (self.alpha * second_term)
    self.output = self.energy
    return self.energy

# Network Construction


In [60]:
def generate_neurones(puzzle, size):
  neurones = []
  for i in range(size):
    for j in range(size):
      for k in range(size):
        neurones.append(Neuron(i, j, k, puzzle[i][j][k], size))
  return neurones

def get_index(row, column, number, size):
  return int((column * (size ** 2)) + (row * size) + number)

def step(neurones, size):
  for _ in range(2):  # the first iteration fills the grid with possibilities, and the second drops the energy of ones it dislikes.

    outputs = [[[0] * size for _ in range(size)]for _ in range(size)]       # stores the outputs from each neurone after each step

    for i in range(size):
      for j in range(size):
        for k in range(size):
          outputs[i][j][k] = neurones[get_index(i,j,k,size)].output

    for itt in range(0, size**3):
      neurones[itt].step_neurone(outputs)

  return neurones

def build_board(neurones, puzzle, size):
  for i in range(size):
    for j in range(size):
      k_to_set = 0
      lowest_k = -1
      for k in range(size):
        if neurones[get_index(i,j,k,size)].output < lowest_k or lowest_k == -1:
          lowest_k = neurones[get_index(i,j,k,size)].output
          k_to_set = k
      for k in range(size):
        puzzle[i][j][k] = 0
      puzzle[i][j][k_to_set] = 1      
  return puzzle

def is_violation(x, y, z, board, size):
  for t in range(size):
    if t != x:
      if board[t][y] == z:
        return True
    if t != y:
      if board[x][t] == z:
        return True
  return False

def reset_network(board, size, original):
  for x in range(size):
    for y in range(size):
      if original[x][y] == 0:
        if is_violation(x, y, board[x][y], board, size):
          board[x][y] = 0
  
  neurones = generate_neurones(network_format(board, size), size)   
  return neurones

In [62]:
def run_network(sudoku, size, steps):
  puzzle = network_format(sudoku, size)
  neurones = generate_neurones(puzzle, size)

  for x in range(steps):
    neurones = step(neurones, size)

    neurones = reset_network(readable_format(build_board(neurones, puzzle, size),size), size, sudoku)
  
  neurones = step(neurones, size)  

  return(readable_format(build_board(neurones, puzzle, size),size))

In [64]:
run_network(trivial, 9, 1)

[[8, 9, 1, 2, 7, 4, 5, 6, 3],
 [6, 2, 3, 1, 8, 5, 9, 4, 1],
 [4, 5, 7, 6, 3, 9, 1, 8, 2],
 [5, 8, 6, 4, 1, 7, 2, 3, 9],
 [7, 4, 2, 9, 5, 3, 8, 1, 6],
 [3, 1, 9, 8, 2, 6, 7, 5, 4],
 [9, 3, 8, 5, 4, 2, 1, 7, 6],
 [1, 6, 4, 7, 9, 8, 3, 2, 5],
 [2, 7, 5, 3, 6, 1, 4, 9, 8]]